In [1]:
import pandas as pd
from data_definition import load_and_transform, create_demand
import pyomo.environ as pyo
from multiprocessing import Pool

# Data import

In [2]:
nodes, channels = load_and_transform("../data/network_graph_2024_06_12.json")
nodes = create_demand(nodes)

Transaction of 15869 sats from PlebsUnited to 02fc1f8c244c9d275f99.


In [3]:
## TODO: parallelization of channel finding and listing for every node
## In order to accomplish that, the nodes dataset is split into 6 parts,
## then the execution of the finding function is parallelized for every
## dataframe slice.
## TODO: Process executed before creation of directed edges in the channels dataframe

def nodes_df_splitting(pd_object: pd.DataFrame, n: int) -> list:
    results = []
    splitting: int = len(pd_object) // n
    for i in range(n):
        ris = pd_object[i * splitting : (i+1) * splitting]
        results.append(ris)
    if len(pd_object) % n != 0:
        ris = pd_object[n * splitting: len(pd_object)]
        results.append(ris)
    return results

In [4]:
slices = 10
dataframes = nodes_df_splitting(nodes, slices)

In [5]:
def find_channels(n: str) -> list:
    """
    :param n: node pub key
    :return: list of channels for the node
    Note that the listed channels are directed channels
    that have also a mirrored channel that describes the
    flow of funds in the opposite direction.
    Thus, this channels list is simply adapted to other
    flow of channels by considering then the channels
    with id "INV-<channel_id>"
    """
    channels_list = []
    for c in channels.index:
        if channels.loc[c, "node1_pub"] == n.name:
            channels_list.append(c)
    return channels_list


def parallel_channel_finding(args: tuple) -> pd.DataFrame:
    dfs, i = args
    df = dfs[i].copy()
    df["outgoing_channels"] = df.apply(find_channels, axis=1)
    return df


pool = Pool() # to set custom processes use processes=4
inputs: list = [(dataframes, y) for y in range(slices)]
outputs: list = pool.map(parallel_channel_finding, inputs)


In [8]:
#nodes["outgoing_channels"] = nodes.apply(find_channels, axis=1)
outputs[0]


,alias,addresses,demand,outgoing_channels
pub_key,,,,
0200000000727d3b67513c916f16975e3bf8f3304cf3fcf0ed855e2ae41888f461,lightningspore.com,1,0,[914492408312299520]
0200000000a3eff613189ca6c4070c89206ad658e286751eca1f29262948247a5f,pay.lnrouter.app,2,0,[916659545733136385]
020003b9499a97c8dfbbab6b196319db37ba9c37bccb60477f3c867175f417988e,BJCR_BTCPayServer,2,0,"[878783569160110081, 879850095339962369, 87999..."
0200420ebfb0c6162e7dfcbcb5ca23df9f423ceb0536617d1b4549fc4efa6a15d5,Dezeland,1,0,"[918121896067661824, 921152150182100993, 92594..."
020045e9b463e642225bc147b77b8ee5d7cc356a42cd18cc67fe37b75c4611ad14,Test0,1,0,[907297204189528064]
...,...,...,...,...
0234d43e3296dcf4934819f49c467c0d703f1e7b6fc528966795a4a7b6640832ce,danielflora,1,0,[919348951086923776]
0234d65a1f6764d8a5fe3fccd32c9176c7dc1d8d4bd741dd181c1dc0f584eb1a1c,irunbtc,1,0,[909883255541137409]
0234e82dea741df5f43a54fc1fc0bf850e799ee3f060e4031d760b3a90399523d2,bip39cups,1,0,[885503784091779073]


## Modeling

In [ ]:
model = pyo.ConcreteModel(name="Min cost flow problem")
model.NODES = pyo.Set(initialize=nodes.index)
model.CHANNELS = pyo.Set(initialize=channels.index) #within=model.NODES*model.NODES)

In [ ]:
model.x = pyo.Var(model.CHANNELS, domain=pyo.Binary)
model.a = pyo.Var(model.CHANNELS, domain=pyo.NonNegativeReals)

In [ ]:
def objective_function(model: pyo.ConcreteModel):
    return sum(channels.loc[k, "base_fee"] * model.x[k] for k in model.CHANNELS) + sum(channels.loc[i, "rate_fee"] * model.a[i] for i in model.CHANNELS)

model.totalCost = pyo.Objective(expr=objective_function(model), sense=pyo.minimize)

### Constraints

#### Capacity constraint

$$amount_{i,j} \le capacity_{i,j} \times x_{i,j} \text{ } \forall (i,j) \in E$$


In [ ]:
def capacity_constraint(model: pyo.ConcreteModel, a):
    return model.a[a] <= channels.loc[a, "capacity"] * model.x[a]

model.CapacityConstraint = pyo.Constraint(model.CHANNELS, rule=capacity_constraint, name="Capacity constraint")

#### Flow balance constraint

$$\sum_{(s,i) \in E} amount_{si} - \sum_{(i,t) \in E} amount_{it} = b_i \text{ } \forall i \in V$$


In [ ]:
#def compute_outgoing(n: str) -> list:
#    """
#    Compute outgoing channels list for the node n
#    :param n: node identifier
#    :return: list of outgoing channels for node n
#    """
#    return [c for c in model.CHANNELS if channels.loc[c, "node1_pub"] == n]
#
#
#def compute_incoming(n: str) -> list:
#    """
#    Compute incoming channels list for the node n
#    :param n: node identifier
#    :return: list of incoming channels for node n
#    """
#    return [c for c in model.CHANNELS if channels.loc[c, "node2_pub"] == n]


def flow_balance_constraint(model: pyo.ConcreteModel, n: str):
    print("Start processing")
    return sum(model.a[a] for a in compute_incoming(f"{n}")) - sum(model.a[a] for a in compute_outgoing(f"{n}")) == nodes.loc[n, "demand"]


model.FlowBalanceConstraint = pyo.Constraint(model.NODES, rule=flow_balance_constraint, name="Flow balance constrain")

## Solving the model

In [ ]:
opt = pyo.SolverFactory('cbc')
opt.solve(model, tee=True)

In [ ]:
model.x.pprint()